In [98]:
from pynq import Overlay
import numpy as np
from pynq import MMIO
import os 
from pynq import allocate
import math
import pynq.lib.dma
import time
from pynq import PL
import asyncio
from pynq import Interrupt

# 导入硬件工程

In [99]:
overlay = Overlay("/home/xilinx/pynq/overlays/base/Accelerator/Accelerator_number.bit")
CNN_Control=overlay.CNN_ACCELERATOR.CNN_Control_1 
Windows_Convert=overlay.CNN_ACCELERATOR.Windows_Data_Convert_0
dma_result=overlay.axi_dma_result       
dma_picture=overlay.axi_dma_picture_window 
dma_weight=overlay.axi_dma_weight      
dma_weight_fc=overlay.axi_dma_weight_full 

CONV_DATA_SEND = dma_picture.sendchannel
CONV_WEIGHT_SEND = dma_weight.sendchannel

POOL_DATA_SEND = dma_picture.sendchannel

FC_DATA_SEND = dma_weight.sendchannel
FC_WEIGHT_SEND = dma_weight_fc.sendchannel

RESULT_RECV= dma_result.recvchannel


Interrupt_Control_address=0x4120_0000
Interrupt_Control_range=0xFFF
Interrupt_mmio=MMIO(Interrupt_Control_address,Interrupt_Control_range)
Interrupt_Data_address=0x0000
Interrupt_Enable_address=0x0128
Interrupt_GEnable_address=0x011c
Interrupt_state_address=0x0120
Interrupt_mmio.write(Interrupt_Enable_address,0x1)
Interrupt_mmio.write(Interrupt_GEnable_address,0x1)
Interrupt_mmio.write(Interrupt_Enable_address,0x1)

In [100]:
def Get_Interrupt_settings():
    CNN_Control_mmio.write(cnn_start_address  ,1)
#     while(Interrupt_mmio.read(Interrupt_state_address)==0): 
#         Interrupt_Enable_address=0x0128
#     end=time.time()
#     cost_time=round(end-start,5)
#     Interrupt_mmio.write(Interrupt_state_address,0x1)  #清除标志
#     print('-------------time cost is--------------- '+ str(cost_time)+'s')
        

# 地址定义

In [101]:
#-------------------------------------------CNN Control模块-----------------------------------------
CNN_Control_address=0x4000_0000
CNN_Control_range=0xFFF
CNN_Control_mmio=MMIO(CNN_Control_address,CNN_Control_range)

#---------------CNN寄存器地址------------------
Conv_Kernel_Size_address=0x0;
Conv_Kernel_Num_address=0x4;
Pool_Kernel_Size_address=0x8;
Pool_Kernel_Num_address=0xc;
FULLCON_Input_Size_address=0x10;
FULLCON_Output_Size_address=0x14;
select_mode_address=0x18;
cnn_start_address=0x1c;
#---------------CNN mode------------------
MODE_IDLE      =0b00000
MODE_CONV      =0b00001
MODE_RELU      =0b00010
MODE_POOL_Max  =0b00100
MODE_POOL_Mean =0b00101
MODE_FULLCON   =0b01000
MODE_CONV_RELU =0b10000

# #---------------寄存器参量------------------
# Conv_Kernel_Size   =4;
# Conv_Kernel_Num    =4;
# Pool_Kernel_Size   =4;
# Pool_Kernel_Num    =1;
# FULLCON_Input_Size =28;
# FULLCON_Output_Size=8;
# select_mode        =MODE_CONV;
# cnn_start          =0;

#-------------------------------------------滑动窗口-----------------------------------------
Windows_Data_address=0x4000_1000
Windows_Data_range=0xFFF
Windows_mmio=MMIO(Windows_Data_address,Windows_Data_range)
Data_horizontal_address=0x0;
Data_vertical_address=0x4;
Kernel_size_address=0x8;
Stride_address=0xc;
Refresh_address=0x10;

# Data_vertical  =28
# Data_horizontal=28
# Stride          =2
# Refresh         =0

In [102]:
def Sliding_window_settings(Data_h,Data_v,Kernel_size,Stride=2,Refresh=0):
    Windows_mmio.write(Kernel_size_address,Kernel_size)
    Windows_mmio.write(Data_vertical_address,Data_v)
    Windows_mmio.write(Data_horizontal_address,Data_h)
    Windows_mmio.write(Stride_address,Stride)
    Windows_mmio.write(Refresh_address,Refresh)
    
#     Kernel_size=Windows_mmio.read(Kernel_size_address)
#     Data_vertical=Windows_mmio.read(Data_vertical_address)
#     Data_horizontal=Windows_mmio.read(Data_horizontal_address)
#     Stride=Windows_mmio.read(Stride_address)
#     Refresh=Windows_mmio.read(Refresh_address)
#     print("滑窗模块 Kernel_size核心大小      =",Kernel_size)
#     print("滑窗模块 Data_vertical图像纵向长  =",Data_vertical)
#     print("滑窗模块 Data_horizontal图像横向长=",Data_horizontal)
#     print("滑窗模块 Stride步进               =",Stride)
#     print("滑窗模块 Refresh刷新              =",Refresh)
  

In [103]:
def Convolutional_settings(mode,Kernel_Size,Kernel_Num,Stride,conv_weight_buffer,Data_h,Data_v,input_picture_buffer,Weight_refresh=1):
    #滑动窗设置
    Conv_input_data_vertical   = Data_v 
    Conv_input_data_horizontal = Data_h
    Conv_Stride =Stride
    Conv_Kernel_Size = Kernel_Size
    Conv_Kernel_Num = Kernel_Num

    Sliding_window_settings(Conv_input_data_horizontal,Conv_input_data_vertical,Conv_Kernel_Size,Conv_Stride,0)
    
    #CNN 模块设置
    if(mode==MODE_CONV):
        selectmode=MODE_CONV
#         print ( " CNN计算模式  :CONV   ")
    else :
        selectmode=MODE_CONV_RELU
#         print ( " CNN计算模式  :CONV_RELU   ")
    CNN_Control_mmio.write(select_mode_address        ,selectmode        )
    CNN_Control_mmio.write(Conv_Kernel_Size_address   ,Conv_Kernel_Size   )
    CNN_Control_mmio.write(Conv_Kernel_Num_address    ,Conv_Kernel_Num    )
#     print ( " 卷积核心大小    ",   Conv_Kernel_Size    )
#     print ( " 卷积核心个数    ",   Conv_Kernel_Num     ) 
    
#     print("--------------------WEIGHT--------------------")
#     for i in range(Kernel_Size*Kernel_Size):
#             print(hex(conv_weight_buffer[i]))   
    
    
    
    
    #----------------------------传递权重----------------
    if(Weight_refresh==1):
        CONV_WEIGHT_SEND.transfer(conv_weight_buffer)
        CONV_WEIGHT_SEND.wait()
#     print("--------------------PICTURE--------------------")
#     for i in range(Conv_input_data_horizontal ):
#         for j in range(Conv_input_data_vertical):
#             print(hex(input_picture_buffer[i*Conv_input_data_vertical+j])," ",end="")
#         print ("\n")       
        
        
        
    #----------------------------传递激励数据----------------
    CONV_DATA_SEND.transfer(input_picture_buffer)
    CONV_DATA_SEND.wait()
    
    #-----------------------------开始计算，并记录时间-------------------------
    
    CNN_Control_mmio.write(cnn_start_address  ,0)
    Get_Interrupt_settings()

    #------------------------------结果--------------------------   
    if(selectmode==MODE_CONV ):

        CONV_hang_size=math.floor((Conv_input_data_horizontal-Conv_Kernel_Size)/Conv_Stride+1)
#         print("结果输出一行个数---------",CONV_hang_size)
        result_data_size=CONV_hang_size*CONV_hang_size
#         print("结果输出总个数---------",result_data_size)
        result_data_size=math.ceil(result_data_size) 
        result_buffer = allocate(shape=(result_data_size,8), dtype=np.int32)
        #-------------------------------获取计算结果-------------------------
        RESULT_RECV.transfer(result_buffer)
        RESULT_RECV.wait() 
        matrix_conv = np.zeros((Conv_Kernel_Num, result_data_size), dtype=np.int32)
        matrix_conv=result_buffer.T[0:Conv_Kernel_Num][:]

        
#         for h in range(Conv_Kernel_Num):
#             print("-------------------第", h+1,"个卷积算子结果------------------","  ", end="")#
#             for j in range(int(result_data_size)):
#                 if(h<Conv_Kernel_Num):
#                     if(j%CONV_hang_size==0):
#                          print("\n")   
#                     print((result_buffer.T[h][j]),"  ", end="")
#             print("")    

    if( selectmode==MODE_CONV_RELU    ):
        
        CONV_hang_size=math.floor((Conv_input_data_horizontal-Conv_Kernel_Size)/Conv_Stride+1)
#         print("结果输出一行个数---------",CONV_hang_size)
        result_data_size=CONV_hang_size*CONV_hang_size
#         print("结果输出总个数---------",result_data_size)
        result_data_size=math.ceil(result_data_size) 
        result_buffer = allocate(shape=(result_data_size,32), dtype=np.int8)
        #-------------------------------获取计算结果-------------------------
        RESULT_RECV.transfer(result_buffer)
        RESULT_RECV.wait() 
        matrix_conv = np.zeros((Conv_Kernel_Num, result_data_size), dtype=np.int8)
        matrix_conv=result_buffer.T[0:Conv_Kernel_Num][:]

        
#         for h in range(Conv_Kernel_Num):
# #             print("-------------------第", h+1,"个卷积算子结果------------------","  ", end="")#
#             for j in range(int(result_data_size)):
#                 if(h<Conv_Kernel_Num):
#                     if(j%CONV_hang_size==0):
#                          print("\n")    
#                     print((matrix_conv[h][j]),"  ", end="")
#             print("")    
    return matrix_conv


In [104]:
def Pooling_settings(mode,Data_horizontal,Data_vertical,input_picture_buffer,Kernel_Size,Kernel_Num=1):
   #滑动窗设置
    Pool_input_data_vertical =Data_vertical  
    Pool_input_data_horizontal =Data_horizontal
    Pool_Kernel_Size =Kernel_Size
    Pool_Kernel_Num  =Kernel_Num
    Pool_Stride      =Kernel_Size
    
    Sliding_window_settings(Pool_input_data_horizontal,Pool_input_data_vertical,Pool_Kernel_Size,Pool_Stride,0)
    if(mode==MODE_POOL_Max):
        selectmode=MODE_POOL_Max
    else :
        selectmode=MODE_POOL_Mean
    CNN_Control_mmio.write(select_mode_address        ,selectmode        )
    CNN_Control_mmio.write(Pool_Kernel_Size_address   ,Pool_Kernel_Size   )
    CNN_Control_mmio.write(Pool_Kernel_Num_address    ,Pool_Kernel_Num    )
#     print ( " 卷积核心大小    ",   Pool_Kernel_Size    )
#     print ( " 卷积核心个数    ",   Pool_Kernel_Num     ) 

    #----------------------------传递激励数据----------------
    CONV_DATA_SEND.transfer(input_picture_buffer)
    CONV_DATA_SEND.wait()
    
    #-----------------------------开始计算，并记录时间-------------------------
    
    CNN_Control_mmio.write(cnn_start_address  ,0)
    Get_Interrupt_settings()
    
    #---------------------------结果--------------------------        
    if(mode==MODE_POOL_Max or mode==MODE_POOL_Mean ):
#         if(mode==MODE_POOL_Max):
#             print ( " CNN计算模式   :POOL_Max    ")
#         else :
#             print ( " CNN计算模式   :POOL_Mean    ")
        Pool_hang_size=math.floor((Pool_input_data_horizontal-Pool_Kernel_Size)/Pool_Kernel_Size+1)
        Pool_data_size=Pool_hang_size*Pool_hang_size
        result_data_size=Pool_data_size*32#多的空闲位置  16bit*8
        matrix_pool = np.zeros((Pool_data_size,), dtype=np.int8)
        result_data_size=math.ceil(result_data_size)
        result_buffer = allocate(shape=(result_data_size,), dtype=np.int8)
#         print("结果输出个数---------",result_data_size)
        RESULT_RECV.transfer(result_buffer)
        RESULT_RECV.wait()
        for j in range(Pool_data_size):
               matrix_pool[j]=result_buffer[j*32]    
#         print("------------------- 池化算子结果------------------",  "\n")
#         for j in range(Pool_data_size):
#             if(j%Pool_hang_size==0):
#                  print("\n")
#             print(( matrix_pool[j]),"  ", end="")
#         print("\n")    
        
    
    
    
    return matrix_pool

In [105]:
def Fullconnection_settings(mode,Input_Size,Output_Size,FC_weight_buffer,FC_act_buffer):
    #滑动窗设置
    global FULLCON_Input_Size   
    global FULLCON_Output_Size  
    global start
    FULLCON_Input_Size    =Input_Size  
    FULLCON_Output_Size   =Output_Size
    #CNN 模块设置
    if(mode==MODE_FULLCON):
        selectmode=MODE_FULLCON
    else :
        selectmode=MODE_FULLCON
    CNN_Control_mmio.write(select_mode_address        ,selectmode        )

    #----------------------------传递权重----------------
    FC_WEIGHT_SEND.transfer(FC_weight_buffer)
    FC_WEIGHT_SEND.wait()
    #----------------------------传递激励数据----------------
    FC_DATA_SEND.transfer(FC_act_buffer)
    FC_DATA_SEND.wait()
    
    #-----------------------------开始计算，并记录时间-------------------------
    
    CNN_Control_mmio.write(cnn_start_address  ,0)
    Get_Interrupt_settings()
    
    
    #--------------------------------------结果---------------------------   
#     print ( " CNN计算模式   :FULLCON  ")
    result_data_size=8   #多的空闲位置  32bit*8
    result_data_size=math.ceil(result_data_size) 
    result_buffer = allocate(shape=(result_data_size,), dtype=np.int32)
    #-------------------------------获取计算结果-------------------------
    RESULT_RECV.transfer(result_buffer)
    RESULT_RECV.wait() 
    data =np.zeros((FULLCON_Output_Size,))
#     print("------------------- 全连接算子结果------------------",  "\n")
    for j in range(FULLCON_Output_Size):
        data[j]=result_buffer[j]
#         print(( result_buffer[j]),"  ", end="")
#     print("\n")

    return data

# 卷积输入激励数据 权重数据

In [106]:
from PIL import Image
# 读取图像并将其转换为灰度图
def read_and_convert_image(filename):
    image = Image.open(filename).convert('L')  # 'L'表示将图像转换为灰度图
    return image

# 将图像数据存储到PYNQ分配的内存空间中
def store_image_data(image,buffer):
    width, height = image.size
    for y in range(height):
        for x in range(width):
            # 获取像素值并存储到分配的内存空间中
            buffer[y*width+x] = image.getpixel((x, y))

In [107]:
# 读取TXT文件
def read_txt_file(filename):
    with open(filename, 'r') as file:
        data = file.readlines()
    return data

# 将文本数据转换为整数数组
def store_txt_data(text_data,buffer):
    parsed_data = []
    for line in text_data:
        # 假设每行数据以空格分隔，并且数据都是整数
        parsed_line = [int(x,16) for x in line.split()]
        parsed_data.append(parsed_line)
    
    # 获取数据的维度
    rows = len(parsed_data)
    cols = len(parsed_data[0])
    # 将数据复制到分配的内存空间中
    for i in range(rows):
        for j in range(cols):
            buffer[i*cols+j]=parsed_data[i][j]   
    return    
def read_8bit_binary_data(file_path):
    with open(file_path, 'r') as file:
        # 读取文件的每一行
        lines = file.readlines()

    # 初始化一个空列表来存储转换后的整数
    parsed_data = []

    # 遍历每一行
    for line in lines:
        # 去除行尾的换行符并转换为整数
        # 假设每行都是有效的8位二进制数，并且没有前导空格
        parsed_data.append(int(line.strip(), 2))

    return parsed_data

def read_8bit_binary_data(file_path):
    with open(file_path, 'r') as file:
        # 读取文件的每一行
        lines = file.readlines()

    # 初始化一个空列表来存储转换后的整数
    parsed_data = []

    # 遍历每一行
    for line in lines:
        # 去除行尾的换行符并转换为整数
        # 假设每行都是有效的8位二进制数，并且没有前导空格
        parsed_data.append(int(line.strip(), 2))

    return parsed_data
# 假设我们有一个函数来将数据存储到buffer中
def store_image_buffer(data, buffer):
    # 将数据复制到buffer中
    for i, value in enumerate(data):
        buffer[i] = value

In [108]:
import matplotlib.image as mpimg # mpimg 用于读取图片
import numpy as np
number=0
count=0
path0 = 'number/'+str(number)+'/' # 输入图片存储目录
path1 = r'number_txt/' # 输出 txt 存储目录
size = 28 # 输入图片大小 28*28

# 十进制转二进制，8bit
def dec2bin(dec_num, bit_wide=8):
    _, bin_num_abs = bin(dec_num).split('b')
    if len(bin_num_abs) > bit_wide:
        raise ValueError           # 数值超出bit_wide长度所能表示的范围
    else:
        if dec_num >= 0:
            bin_num = bin_num_abs.rjust(bit_wide, '0')
        else:
            _, bin_num = bin(2**bit_wide + dec_num).split('b')
    return bin_num
# 读取.png文件，此时pic为一个np.array数组，每个像素值为0-1
pic = mpimg.imread(path0 + str(count)+'.png')
# 转为 8bit数据  0-12w
pic = pic*127
pic = pic.astype(int) # float -> int

f = open(path1 + str(number)+"_"+str(count)+".txt",'w')
for i in range (size):
    for j in range(size):
        f.write(dec2bin(pic[i][j]))
        f.write("\n") # 一行一个8bit像素数据

f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'number/0/0.png'

In [ ]:
# filename =path1 + str(number)+"_"+str(count)+".txt"
filename="number_txt/6_6.txt"
#2_2.txt   5_2
text_data = read_8bit_binary_data(filename)


width, height = 28,28
CONV1_DataV=width
CONV1_DataH=CONV1_DataV
Conv1_Kernel_Size=5
Conv1_Stride=1
Conv1_Kernel_Num=6
Conv1_mode=MODE_CONV_RELU
Conv1_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV1_data_size = CONV1_DataH*CONV1_DataV
CONV1_in_buffer = allocate(shape=(CONV1_data_size,), dtype=np.int8) 
store_image_buffer(text_data,CONV1_in_buffer)    
    
print("-------------------------ACT  Show ---------------------") 
for j in range(CONV1_DataH):
    print("\n")
    for k in range(CONV1_DataV):
        print((CONV1_in_buffer[j*CONV1_DataH+k]),"  ", end="")
print("\n")       

#------------------------------设置权重---------------------------------
Conv1_weight_buffer = allocate(shape=(Conv1_Kernel_Size*Conv1_Kernel_Size,), dtype=np.uint64)

 ## 读取TXT文件
filename = 'Lenet_new_weight/Conv1_weight0combine6_0.txt'
text_data = read_txt_file(filename)

# 存文本数据
store_txt_data(text_data,Conv1_weight_buffer)

for j in range(Conv1_Kernel_Size*Conv1_Kernel_Size ):
    print(hex(Conv1_weight_buffer[j]),"  ", end="")
    print("-------------------", j,  "\n")
    
    

In [109]:
CONV1_result_hang=math.floor((CONV1_DataH-Conv1_Kernel_Size)/Conv1_Stride+1)
Conv1_result_data_size=CONV1_result_hang*CONV1_result_hang
Conv1_result_data_size=math.ceil(Conv1_result_data_size) 
conv1_result = np.zeros((Conv1_Kernel_Num, Conv1_result_data_size), dtype=np.int8)
start=time.time()
conv1_result=Convolutional_settings(Conv1_mode,Conv1_Kernel_Size,Conv1_Kernel_Num,Conv1_Stride,Conv1_weight_buffer,CONV1_DataH,CONV1_DataV,CONV1_in_buffer,Conv1_Weight_refresh)


# 池化层参数输入

In [110]:
Pool1_mode=MODE_POOL_Max
#---------------------一个输入的池化大小尺寸----------------
Pool1_DataH=CONV1_result_hang
Pool1_DataV=Pool1_DataH
Pool1_Kernel_Size=2
Pool1_data_size = Pool1_DataH*Pool1_DataV
Pool1_buffer = allocate(shape=(Pool1_data_size,), dtype=np.int8)

#----------------------------池化输出结果-----------------------------
Pool1_result_hang_size=math.floor((Pool1_DataH-Pool1_Kernel_Size)/Pool1_Kernel_Size+1)
Pool1_result_data_size=Pool1_result_hang_size*Pool1_result_hang_size
pool1_result = np.zeros((Conv1_Kernel_Num,Pool1_result_data_size), dtype=np.int8)

for j in range (Conv1_Kernel_Num):
    for i in range(Pool1_data_size):
        Pool1_buffer[i] =conv1_result[j][i]
    start=time.time()
    pool1_result[j]=Pooling_settings(Pool1_mode,Pool1_DataH,Pool1_DataV,Pool1_buffer,Pool1_Kernel_Size,1)
    print("done",j)

done 0
done 1
done 2
done 3
done 4
done 5


#  第二层卷积

In [111]:
# input_picture_buffer 初始化
CONV2_DataV=Pool1_result_hang_size
CONV2_DataH=CONV2_DataV
Conv2_Kernel_Size=5
Conv2_Stride=1
Conv2_Kernel_Num=6
Conv2_mode=MODE_CONV
Conv2_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV2_data_size = CONV2_DataH*CONV2_DataV
CONV2_in_buffer = allocate(shape=(CONV2_data_size,), dtype=np.int8)

#-----------------------------权重数据-------------------------------
Conv2_weight_buffer1 = allocate(shape=(Conv2_Kernel_Size*Conv2_Kernel_Size,), dtype=np.int64)




Conv2_weight_buffer2 = allocate(shape=(Conv2_Kernel_Size*Conv2_Kernel_Size,), dtype=np.int64)


#-----------------------计算结果---------------------------
CONV2_result_hang=math.floor((CONV2_DataH-Conv2_Kernel_Size)/Conv2_Stride+1)
Conv2_result_data_size=CONV2_result_hang*CONV2_result_hang
Conv2_result_data_size=math.ceil(Conv2_result_data_size) 
conv2_result1 = np.zeros((Conv2_Kernel_Num, Conv2_result_data_size), dtype=np.int32)
conv2_result2 = np.zeros((Conv2_Kernel_Num, Conv2_result_data_size), dtype=np.int32)


# # print(CONV2_data_size)
for i in range(Conv2_Kernel_Num):#6个通道
    for n in range(CONV2_data_size):
        CONV2_in_buffer[n]=pool1_result[i][n]   #更新输入数据   
    # 读取TXT文件
    filename1 = 'Lenet_new_weight/Conv2_weight'+str(i)+'0combine6_0.txt'
    text_data1 = read_txt_file(filename1)
    # 存文本数据
    store_txt_data(text_data1,Conv2_weight_buffer1)  
    for j in range(Conv2_Kernel_Size*Conv2_Kernel_Size ):
        print(hex(Conv2_weight_buffer1[j]),"  ", end="")
        print("--------1-----------", j,  "\n")
        # 读取TXT文件
    filename2 = 'Lenet_new_weight/Conv2_weight'+str(i)+'1combine6_0.txt'
    text_data2 = read_txt_file(filename2)
    # 存文本数据
    store_txt_data(text_data2,Conv2_weight_buffer2)

    
    start=time.time()    
    conv2_result1+=Convolutional_settings(Conv2_mode,Conv2_Kernel_Size,Conv2_Kernel_Num,Conv2_Stride,Conv2_weight_buffer1,CONV2_DataH,CONV2_DataV,CONV2_in_buffer,Conv2_Weight_refresh)
    for j in range(Conv2_Kernel_Size*Conv2_Kernel_Size ):
        print(hex(Conv2_weight_buffer2[j]),"  ", end="")
        print("--------2-----------", j,  "\n")
    start=time.time()  
    conv2_result2+=Convolutional_settings(Conv2_mode,Conv2_Kernel_Size,Conv2_Kernel_Num,Conv2_Stride,Conv2_weight_buffer2,CONV2_DataH,CONV2_DataV,CONV2_in_buffer,Conv2_Weight_refresh)

    

0xde2e34db0924   --------1----------- 0 

0xd8040ae3123b   --------1----------- 1 

0xe62ee21f1ff3   --------1----------- 2 

0xe91dc618110b   --------1----------- 3 

0xe018da1e1607   --------1----------- 4 

0xe1ff2fb2c045   --------1----------- 5 

0x5183310a915   --------1----------- 6 

0xd41bd303c2c3   --------1----------- 7 

0xcd29ea0acb0b   --------1----------- 8 

0xb834fe0b05fd   --------1----------- 9 

0x10fd31fffdea   --------1----------- 10 

0x13053207d8ca   --------1----------- 11 

0x39260612d9bb   --------1----------- 12 

0xf23eeee7ea0e   --------1----------- 13 

0xd123e406de04   --------1----------- 14 

0x21de1b0416ba   --------1----------- 15 

0x2c3311e09d4   --------1----------- 16 

0x24e212010ce4   --------1----------- 17 

0x1bdb0def122c   --------1----------- 18 

0x2403e8042911   --------1----------- 19 

0xc8e1f91119ec   --------1----------- 20 

0xd9c01de81803   --------1----------- 21 

0xe3f207021520   --------1----------- 22 

0x14d206f93329   ------

In [112]:
# for i in range(conv2_result1.shape[0]):
#     for j in range(conv2_result1.shape[1]):
#         print(conv2_result1[i][j],"  " ,end ="   ")
#         if(j%8==7):
#             print("\n")
#     print("\n")

In [113]:
# for i in range(conv2_result2.shape[0]):
#     for j in range(conv2_result2.shape[1]):
#         print(conv2_result2[i][j],"  " ,end ="   ")
#         if(j%8==7):
#             print("\n")
#     print("\n")

In [114]:
for i in range(conv2_result1.shape[0]):
    for j in range(conv2_result1.shape[1]):
        if(conv2_result1[i][j]<0):
            conv2_result1[i][j]=0
        else:
            conv2_result1[i][j]=math.ceil(conv2_result1[i][j]>>10)#
        print(conv2_result1[i][j],"  " ,end ="   ")
        if(j%8==7):
            print("\n")
    print("\n")

10      0      0      0      36      52      19      6      

7      0      0      0      53      52      12      2      

0      0      0      0      34      10      0      0      

0      0      0      0      0      0      0      0      

0      0      0      0      0      0      0      0      

0      0      0      0      0      0      0      0      

0      0      0      0      0      0      0      0      

0      0      0      0      0      0      0      3      



0      0      0      0      0      29      22      7      

0      0      0      0      2      37      25      10      

0      0      0      0      0      0      0      0      

0      0      0      0      0      0      0      0      

0      0      0      0      0      0      0      0      

0      0      0      0      0      0      0      0      

10      7      0      0      0      0      0      0      

15      22      22      16      0      0      0      0      



0      21      33      17      2      6      5   

In [115]:
for i in range(conv2_result2.shape[0]):
    for j in range(conv2_result2.shape[1]):
        if(conv2_result2[i][j]<0):
            conv2_result2[i][j]=0
        else:
            conv2_result2[i][j]=math.ceil(conv2_result2[i][j]>>10)
        print(conv2_result2[i][j],"  " ,end ="   ")
        if(j%8==7):
            print("\n")
    print("\n")

5      1      3      0      0      0      0      0      

7      2      1      0      0      0      0      0      

11      5      1      7      8      6      1      0      

9      4      18      46      41      16      2      1      

10      16      28      44      24      2      2      5      

13      23      22      20      22      30      27      13      

9      15      18      35      53      46      20      3      

2      6      15      30      36      21      5      5      



0      4      7      20      26      17      12      8      

1      7      18      38      50      49      42      31      

0      5      32      63      80      82      70      53      

0      4      14      20      42      70      72      62      

0      0      0      0      0      31      45      52      

0      0      0      0      11      7      4      20      

0      0      0      0      0      0      0      0      

0      0      0      0      0      0      0      0      



0      0     

In [116]:
print(conv2_result1.shape)

(6, 64)


# 第二层池化

In [117]:
Pool21_mode=MODE_POOL_Max
#---------------------一个输入的池化大小尺寸----------------
Pool21_DataH=CONV2_result_hang
Pool21_DataV=Pool21_DataH
Pool21_Kernel_Size=2
Pool21_data_size = Pool21_DataH*Pool21_DataV
Pool2_buffer1 = allocate(shape=(Pool21_data_size,), dtype=np.int8)

#----------------------------池化输出结果-----------------------------
Pool21_result_hang_size=math.floor((Pool21_DataH-Pool21_Kernel_Size)/Pool21_Kernel_Size+1)
Pool21_result_data_size=Pool21_result_hang_size*Pool21_result_hang_size
pool2_result1 = np.zeros((Conv2_Kernel_Num,Pool21_result_data_size), dtype=np.int8)

for j in range (Conv2_Kernel_Num):
    for i in range(Pool21_data_size):
        Pool2_buffer1[i] =conv2_result1[j][i]
    start=time.time()
    pool2_result1[j]=Pooling_settings(Pool21_mode,Pool21_DataH,Pool21_DataV,Pool2_buffer1,Pool21_Kernel_Size,1)
    print("done",j)

done 0
done 1
done 2
done 3
done 4
done 5


In [118]:
#----------------------------池化输出结果-----------------------------
Pool22_mode=MODE_POOL_Max
#---------------------一个输入的池化大小尺寸----------------
Pool22_DataH=CONV2_result_hang
Pool22_DataV=Pool22_DataH
Pool22_Kernel_Size=2
Pool22_data_size = Pool22_DataH*Pool22_DataV
Pool2_buffer2 = allocate(shape=(Pool22_data_size,), dtype=np.int8)


Pool22_result_hang_size=math.floor((Pool22_DataH-Pool22_Kernel_Size)/Pool22_Kernel_Size+1)
Pool22_result_data_size=Pool22_result_hang_size*Pool22_result_hang_size
pool2_result2 = np.zeros((Conv2_Kernel_Num,Pool22_result_data_size), dtype=np.int8)

for j in range (Conv2_Kernel_Num):
    for i in range(Pool22_data_size):
        Pool2_buffer2[i] =conv2_result2[j][i]
    start=time.time()
    pool2_result2[j]=Pooling_settings(Pool22_mode,Pool22_DataH,Pool22_DataV,Pool2_buffer2,Pool22_Kernel_Size,1)
    print("done",j)

done 0
done 1
done 2
done 3
done 4
done 5


# 全连接参数输入

In [119]:
fc_data=np.vstack((pool2_result1, pool2_result2))
fc_data=fc_data.flatten()

In [120]:
# input_weightfc_buffer 初始化  一次传入FULLCON_Output_Size个数据
FULLCON_Input_Size=fc_data.shape[0]
FULLCON_Output_Size1=8
FULLCON_Output_Size2=2
FC_weight_buffer1 = allocate(shape=(FULLCON_Input_Size,), dtype=np.uint64)
FC_weight_buffer2 = allocate(shape=(FULLCON_Input_Size,), dtype=np.uint64)

# 读取TXT文件
filename = 'Lenet_new_weight/fc_weight_combine8.txt'
text_data = read_txt_file(filename)
# 存文本数据
store_txt_data(text_data,FC_weight_buffer1)
# 读取TXT文件
filename = 'Lenet_new_weight/fc_weight_combine2.txt'
text_data = read_txt_file(filename)
store_txt_data(text_data,FC_weight_buffer2)

    
FC_whole_act_data_size = FULLCON_Input_Size
FC_act_buffer = allocate(shape=(FC_whole_act_data_size,), dtype=np.int64)

for i in range(FC_whole_act_data_size):
    FC_act_buffer[i] = fc_data[i]

    
# for j in range(FC_whole_act_data_size):
#     print(hex(FC_weight_buffer1[j]),"  ", end="")
#     print("-------------------", j,  "\n")    
# for j in range(FC_whole_act_data_size):
#     print(hex(FC_weight_buffer2[j]),"  ", end="")
#     print("-------------------", j,  "\n")   

In [121]:
mode=MODE_FULLCON
Input_Size=FULLCON_Input_Size
Output_Size=FULLCON_Output_Size1
start=time.time()
fc1=Fullconnection_settings(mode,Input_Size,Output_Size,FC_weight_buffer1,FC_act_buffer)
Output_Size=FULLCON_Output_Size2
start=time.time()
fc2=Fullconnection_settings(mode,Input_Size,Output_Size,FC_weight_buffer2,FC_act_buffer)

In [122]:
import numpy as np
print("测试数字为：    6")
print("测试结果十分类别如下")
def softmax(x):
    """Compute the softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)  # Ensure axis=0 for 1D arrays

fc_result_data= np.concatenate((fc1, fc2), axis=0)
# 计算每个类别的softmax值
class_probabilities = softmax(fc_result_data)

# 将softmax值转换为百分比，并保留两位小数
percentages = np.round(class_probabilities * 100, 2)

# 打印结果，并添加'%'符号
for i, percentage in enumerate(percentages):
    print(f"Class {i}: 原始输出：{fc_result_data[i]}   Softmax:{percentage}%")

测试数字为：    6
测试结果十分类别如下
Class 0: 原始输出：-19574.0   Softmax:0.0%
Class 1: 原始输出：-20036.0   Softmax:0.0%
Class 2: 原始输出：-7059.0   Softmax:0.0%
Class 3: 原始输出：-7726.0   Softmax:0.0%
Class 4: 原始输出：-8057.0   Softmax:0.0%
Class 5: 原始输出：-6869.0   Softmax:0.0%
Class 6: 原始输出：22927.0   Softmax:100.0%
Class 7: 原始输出：-29289.0   Softmax:0.0%
Class 8: 原始输出：-1906.0   Softmax:0.0%
Class 9: 原始输出：-21891.0   Softmax:0.0%


1000张数字测试


In [87]:
import matplotlib.image as mpimg # mpimg 用于读取图片
import numpy as np
import os
import random
rightnumber=np.zeros((10,100))
for number in range(10):
    for count in range(100):
        path0 = "/home/xilinx/pynq/overlays/test_number/"+str(number)+'/' # 输入图片存储目录
        path1 = r'number_txt/' # 输出 txt 存储目录
        size = 28 # 输入图片大小 28*28

        # 十进制转二进制，8bit
        def dec2bin(dec_num, bit_wide=8):
            _, bin_num_abs = bin(dec_num).split('b')
            if len(bin_num_abs) > bit_wide:
                raise ValueError("数值超出bit_wide长度所能表示的范围")
            else:
                if dec_num >= 0:
                    bin_num = bin_num_abs.rjust(bit_wide, '0')
                else:
                    _, bin_num = bin(2**bit_wide + dec_num).split('b')
            return bin_num
        
        # 从文件夹中随机选择一张图片
        file_list = os.listdir(path0)
        selected_file = random.choice(file_list)
        file_path = os.path.join(path0, selected_file)
        
        # 读取.png文件，此时pic为一个np.array数组，每个像素值为0-1
        pic = mpimg.imread(file_path)
        
        # 转为 8bit数据  0-12w
        pic = pic * 127
        pic = pic.astype(int) # float -> int
        
        # 创建输出文件路径
        output_file = os.path.join(path1,  str(number)+"_"+str(count)+".txt")
        
        # 将像素数据写入输出文件
        with open(output_file, 'w') as f:
            for i in range(size):
                for j in range(size):
                    f.write(dec2bin(pic[i][j]))
                    f.write("\n") # 一行一个8bit像素数据
        f.close()
        
        
        filename =path1 + str(number)+"_"+str(count)+".txt"
        text_data = read_8bit_binary_data(filename)
        width, height = 28,28
        CONV1_DataV=width
        CONV1_DataH=CONV1_DataV
        Conv1_Kernel_Size=5
        Conv1_Stride=1
        Conv1_Kernel_Num=6
        Conv1_mode=MODE_CONV_RELU
        Conv1_Weight_refresh=1
        #------------------------------设置输入数据--------------------------
        CONV1_data_size = CONV1_DataH*CONV1_DataV
        CONV1_in_buffer = allocate(shape=(CONV1_data_size,), dtype=np.int8) 
        store_image_buffer(text_data,CONV1_in_buffer)    
            
#         # print("-------------------------ACT  Show ---------------------") 
#         for j in range(CONV1_DataH):
#             print("\n")
#             for k in range(CONV1_DataV):
#                 print((CONV1_in_buffer[j*CONV1_DataH+k]),"  ", end="")
#         print("\n")       
        
        #------------------------------设置权重---------------------------------
        Conv1_weight_buffer = allocate(shape=(Conv1_Kernel_Size*Conv1_Kernel_Size,), dtype=np.uint64)
        
         ## 读取TXT文件
        filename = 'Lenet_new_weight/Conv1_weight0combine6_0.txt'
        text_data = read_txt_file(filename)
        
        # 存文本数据
        store_txt_data(text_data,Conv1_weight_buffer)
        CONV1_result_hang=math.floor((CONV1_DataH-Conv1_Kernel_Size)/Conv1_Stride+1)
        Conv1_result_data_size=CONV1_result_hang*CONV1_result_hang
        Conv1_result_data_size=math.ceil(Conv1_result_data_size) 
        conv1_result = np.zeros((Conv1_Kernel_Num, Conv1_result_data_size), dtype=np.int8)
        start=time.time()
        conv1_result=Convolutional_settings(Conv1_mode,Conv1_Kernel_Size,Conv1_Kernel_Num,Conv1_Stride,Conv1_weight_buffer,CONV1_DataH,CONV1_DataV,CONV1_in_buffer,Conv1_Weight_refresh)
        
        
        
        Pool1_mode=MODE_POOL_Max
        #---------------------一个输入的池化大小尺寸----------------
        Pool1_DataH=CONV1_result_hang
        Pool1_DataV=Pool1_DataH
        Pool1_Kernel_Size=2
        Pool1_data_size = Pool1_DataH*Pool1_DataV
        Pool1_buffer = allocate(shape=(Pool1_data_size,), dtype=np.int8)
        
        #----------------------------池化输出结果-----------------------------
        Pool1_result_hang_size=math.floor((Pool1_DataH-Pool1_Kernel_Size)/Pool1_Kernel_Size+1)
        Pool1_result_data_size=Pool1_result_hang_size*Pool1_result_hang_size
        pool1_result = np.zeros((Conv1_Kernel_Num,Pool1_result_data_size), dtype=np.int8)
        
        for j in range (Conv1_Kernel_Num):
            for i in range(Pool1_data_size):
                Pool1_buffer[i] =conv1_result[j][i]
            start=time.time()
            pool1_result[j]=Pooling_settings(Pool1_mode,Pool1_DataH,Pool1_DataV,Pool1_buffer,Pool1_Kernel_Size,1)
            
            
            
        CONV2_DataV=Pool1_result_hang_size
        CONV2_DataH=CONV2_DataV
        Conv2_Kernel_Size=5
        Conv2_Stride=1
        Conv2_Kernel_Num=6
        Conv2_mode=MODE_CONV
        Conv2_Weight_refresh=1
        #------------------------------设置输入数据--------------------------
        CONV2_data_size = CONV2_DataH*CONV2_DataV
        CONV2_in_buffer = allocate(shape=(CONV2_data_size,), dtype=np.int8)
        
        #-----------------------------权重数据-------------------------------
        Conv2_weight_buffer1 = allocate(shape=(Conv2_Kernel_Size*Conv2_Kernel_Size,), dtype=np.int64)
        
        
        
        
        Conv2_weight_buffer2 = allocate(shape=(Conv2_Kernel_Size*Conv2_Kernel_Size,), dtype=np.int64)
        
        
        #-----------------------计算结果---------------------------
        CONV2_result_hang=math.floor((CONV2_DataH-Conv2_Kernel_Size)/Conv2_Stride+1)
        Conv2_result_data_size=CONV2_result_hang*CONV2_result_hang
        Conv2_result_data_size=math.ceil(Conv2_result_data_size) 
        conv2_result1 = np.zeros((Conv2_Kernel_Num, Conv2_result_data_size), dtype=np.int32)
        conv2_result2 = np.zeros((Conv2_Kernel_Num, Conv2_result_data_size), dtype=np.int32)
        
        
        # # print(CONV2_data_size)
        for i in range(Conv2_Kernel_Num):#6个通道
            for n in range(CONV2_data_size):
                CONV2_in_buffer[n]=pool1_result[i][n]   #更新输入数据   
            # 读取TXT文件
            filename1 = 'Lenet_new_weight/Conv2_weight'+str(i)+'0combine6_0.txt'
            text_data1 = read_txt_file(filename1)
            # 存文本数据
            store_txt_data(text_data1,Conv2_weight_buffer1)  
#             for j in range(Conv2_Kernel_Size*Conv2_Kernel_Size ):
#                 print(hex(Conv2_weight_buffer1[j]),"  ", end="")
#                 print("--------1-----------", j,  "\n")
                # 读取TXT文件
            filename2 = 'Lenet_new_weight/Conv2_weight'+str(i)+'1combine6_0.txt'
            text_data2 = read_txt_file(filename2)
            # 存文本数据
            store_txt_data(text_data2,Conv2_weight_buffer2)
        
            
            start=time.time()    
            conv2_result1+=Convolutional_settings(Conv2_mode,Conv2_Kernel_Size,Conv2_Kernel_Num,Conv2_Stride,Conv2_weight_buffer1,CONV2_DataH,CONV2_DataV,CONV2_in_buffer,Conv2_Weight_refresh)
#             for j in range(Conv2_Kernel_Size*Conv2_Kernel_Size ):
#                 print(hex(Conv2_weight_buffer2[j]),"  ", end="")
#                 print("--------2-----------", j,  "\n")
            start=time.time()  
            conv2_result2+=Convolutional_settings(Conv2_mode,Conv2_Kernel_Size,Conv2_Kernel_Num,Conv2_Stride,Conv2_weight_buffer2,CONV2_DataH,CONV2_DataV,CONV2_in_buffer,Conv2_Weight_refresh)
        
        for i in range(conv2_result1.shape[0]):
            for j in range(conv2_result1.shape[1]):
                if(conv2_result1[i][j]<0):
                    conv2_result1[i][j]=0
                else:
                    conv2_result1[i][j]=math.ceil(conv2_result1[i][j]>>10)#
        for i in range(conv2_result2.shape[0]):
            for j in range(conv2_result2.shape[1]):
                if(conv2_result2[i][j]<0):
                    conv2_result2[i][j]=0
                else:
                    conv2_result2[i][j]=math.ceil(conv2_result2[i][j]>>10)
        
        
        Pool21_mode=MODE_POOL_Max
        #---------------------一个输入的池化大小尺寸----------------
        Pool21_DataH=CONV2_result_hang
        Pool21_DataV=Pool21_DataH
        Pool21_Kernel_Size=2
        Pool21_data_size = Pool21_DataH*Pool21_DataV
        Pool2_buffer1 = allocate(shape=(Pool21_data_size,), dtype=np.int8)
        #----------------------------池化输出结果-----------------------------
        Pool21_result_hang_size=math.floor((Pool21_DataH-Pool21_Kernel_Size)/Pool21_Kernel_Size+1)
        Pool21_result_data_size=Pool21_result_hang_size*Pool21_result_hang_size
        pool2_result1 = np.zeros((Conv2_Kernel_Num,Pool21_result_data_size), dtype=np.int8)
        
        for j in range (Conv2_Kernel_Num):
            for i in range(Pool21_data_size):
                Pool2_buffer1[i] =conv2_result1[j][i]
            start=time.time()
            pool2_result1[j]=Pooling_settings(Pool21_mode,Pool21_DataH,Pool21_DataV,Pool2_buffer1,Pool21_Kernel_Size,1)
        #----------------------------池化输出结果-----------------------------
        Pool22_mode=MODE_POOL_Max
        #---------------------一个输入的池化大小尺寸----------------
        Pool22_DataH=CONV2_result_hang
        Pool22_DataV=Pool22_DataH
        Pool22_Kernel_Size=2
        Pool22_data_size = Pool22_DataH*Pool22_DataV
        Pool2_buffer2 = allocate(shape=(Pool22_data_size,), dtype=np.int8)
        
        
        Pool22_result_hang_size=math.floor((Pool22_DataH-Pool22_Kernel_Size)/Pool22_Kernel_Size+1)
        Pool22_result_data_size=Pool22_result_hang_size*Pool22_result_hang_size
        pool2_result2 = np.zeros((Conv2_Kernel_Num,Pool22_result_data_size), dtype=np.int8)
        
        for j in range (Conv2_Kernel_Num):
            for i in range(Pool22_data_size):
                Pool2_buffer2[i] =conv2_result2[j][i]
            start=time.time()
            pool2_result2[j]=Pooling_settings(Pool22_mode,Pool22_DataH,Pool22_DataV,Pool2_buffer2,Pool22_Kernel_Size,1)
        
        fc_data=np.vstack((pool2_result1, pool2_result2))
        fc_data=fc_data.flatten()
        
        # input_weightfc_buffer 初始化  一次传入FULLCON_Output_Size个数据
        FULLCON_Input_Size=fc_data.shape[0]
        FULLCON_Output_Size1=8
        FULLCON_Output_Size2=2
        FC_weight_buffer1 = allocate(shape=(FULLCON_Input_Size,), dtype=np.uint64)
        FC_weight_buffer2 = allocate(shape=(FULLCON_Input_Size,), dtype=np.uint64)
        
        # 读取TXT文件
        filename = 'Lenet_new_weight/fc_weight_combine8.txt'
        text_data = read_txt_file(filename)
        # 存文本数据
        store_txt_data(text_data,FC_weight_buffer1)
        # 读取TXT文件
        filename = 'Lenet_new_weight/fc_weight_combine2.txt'
        text_data = read_txt_file(filename)
        store_txt_data(text_data,FC_weight_buffer2)
        
            
        FC_whole_act_data_size = FULLCON_Input_Size
        FC_act_buffer = allocate(shape=(FC_whole_act_data_size,), dtype=np.int64)
        
        for i in range(FC_whole_act_data_size):
            FC_act_buffer[i] = fc_data[i]

    
        mode=MODE_FULLCON
        Input_Size=FULLCON_Input_Size
        Output_Size=FULLCON_Output_Size1
        start=time.time()
        fc1=Fullconnection_settings(mode,Input_Size,Output_Size,FC_weight_buffer1,FC_act_buffer)
        Output_Size=FULLCON_Output_Size2
        start=time.time()
        fc2=Fullconnection_settings(mode,Input_Size,Output_Size,FC_weight_buffer2,FC_act_buffer)
        
        
        fc_result_data= np.concatenate((fc1, fc2), axis=0)
        max_index = np.argmax(fc_result_data)
        if(max_index==number):
            rightnumber[number][count]=1
        else:
            rightnumber[number][count]=0


KeyboardInterrupt: 

In [69]:
result=np.zeros((10))
for i in range(10):
    print("数字",i,"正确性")
    for j in range(100):
        result[i]+=rightnumber[i][j]
    print(result[i],"  "  ,end=" ")
    print("\n")

数字 0 正确性
100.0    

数字 1 正确性
100.0    

数字 2 正确性
100.0    

数字 3 正确性
97.0    

数字 4 正确性
99.0    

数字 5 正确性
96.0    

数字 6 正确性
97.0    

数字 7 正确性
98.0    

数字 8 正确性
96.0    

数字 9 正确性
98.0    

